In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model

/Users/didi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/didi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# VGG16模型,权重由ImageNet训练而来,不保留顶层的3个全连接网络
base_model = VGG16(weights='imagenet', include_top=False)

In [63]:
# 自定义一个简单的分类器
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1000, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 预训练模型部分,权重设置为不训练
for layer in base_model.layers:
    layer.trainable = False

# 编译
model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [64]:
# 传入图像，并进行预处理
img_path = './wolf.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)


In [65]:
# 预测
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=5)[0])

Predicted: [('n02114367', 'timber_wolf', 0.10677604), ('n02125311', 'cougar', 0.04431557), ('n01986214', 'hermit_crab', 0.034899224), ('n02442845', 'mink', 0.030031418), ('n04254777', 'sock', 0.029868854)]
